# Stock Analysis and Recommendation Engine

This notebook analyzes stock data for specified symbols (AAPL, MSFT, GOOG) using technical and quantitative methods to generate Buy/Sell/Hold recommendations.

## 1. Setup and Data Ingestion

Import necessary libraries and the custom `InfluxDBHandler` to connect to the database and retrieve data.

In [ ]:
# Necessary imports
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_ta as ta # For technical indicators
from statsmodels.tsa.arima.model import ARIMA # For quantitative analysis
import statsmodels.api as sm # For potential future diagnostics
from sklearn.metrics import accuracy_score # For evaluating quantitative model (optional)
from datetime import datetime, timedelta
from dotenv import load_dotenv
import warnings

# Import our custom handler
from stock_data_ingestion import InfluxDBHandler

# Suppress warnings (e.g., from ARIMA)
warnings.filterwarnings("ignore")

# Load environment variables (.env file should be in the same directory or specify path)
# Ensure your .env file has TOKEN, ORG, BUCKET, URL, and optionally FINNHUB_API_KEY
load_dotenv()

print("Libraries imported.")

Initialize the `InfluxDBHandler` and connect to the database. Then, retrieve the stock data for the desired symbols and time range.

In [ ]:
# Configuration
symbols = ["AAPL", "MSFT", "GOOG"]
# Define time range for analysis (e.g., last 60 days)
end_time = datetime.now()
start_time = end_time - timedelta(days=210)

# Format for InfluxDB query (RFC3339)
start_time_str = start_time.isoformat() + "Z"
end_time_str = end_time.isoformat() + "Z"

# Initialize and connect
influx_handler = InfluxDBHandler()
stock_data = pd.DataFrame() # Initialize empty dataframe
news_data = {} # Initialize empty dict

if influx_handler.connect():
    if influx_handler.test_connection():
        print(f"Retrieving data for {', '.join(symbols)} from {start_time.date()} to {end_time.date()}...")
        # Retrieve data (only interested in stock data for this analysis)
        # Note: retrieve_data returns stock_df, news_data. We ignore news_data here.
        # IMPORTANT: Ensure retrieve_data fetches OHLCV data if needed by indicators.
        # The current stock_data_ingestion.py retrieves only 'close'.
        # You might need to modify retrieve_data in stock_data_ingestion.py
        # to fetch 'open', 'high', 'low', 'close', 'volume' for full TA.
        # For now, we proceed assuming 'close' is sufficient for the demo.
        stock_data, _ = influx_handler.retrieve_data(symbols, start_time_str, end_time_str)

        if not stock_data.empty:
            print("Stock data retrieved successfully:")
            # Ensure data is sorted by time and index is a DatetimeIndex
            stock_data.index = pd.to_datetime(stock_data.index, utc=True)
            stock_data = stock_data.sort_index()
            # Optional: Fill missing values if any (e.g., forward fill)
            stock_data = stock_data.ffill()
            display(stock_data.head()) # Display first few rows in Jupyter
        else:
            print("Warning: No stock data retrieved. Analysis cannot proceed.")
    else:
        print("Error: InfluxDB connection test failed.")
else:
    print("Error: Failed to connect to InfluxDB.")

## 2. Technical Analysis

Calculate and analyze various technical indicators to understand price trends, momentum, and volatility.

### 2.1 Calculate Indicators

We will use the `pandas_ta` library to calculate:
*   **SMA (Simple Moving Average):** 50-day and 200-day
*   **EMA (Exponential Moving Average):** 20-day
*   **RSI (Relative Strength Index):** 14-day period
*   **MACD (Moving Average Convergence Divergence):** Standard parameters (12, 26, 9)
*   **Bollinger Bands:** 20-day SMA with 2 standard deviations

**Note:** `pandas_ta` often works best with columns named 'open', 'high', 'low', 'close', 'volume'. Our current `retrieve_data` function only gets 'close'. We'll proceed with 'close' but acknowledge that some indicators might ideally use OHLC data.

In [ ]:
# Dictionary to store technical data for each symbol
tech_data = {}

if not stock_data.empty:
    for symbol in symbols:
        print(f"Calculating indicators for {symbol}...")
        # Create a copy to avoid modifying the original dataframe directly
        df = stock_data[[symbol]].copy()
        df.rename(columns={symbol: 'close'}, inplace=True) # pandas_ta often uses 'close'

        # Check if 'close' column exists and is not empty
        if 'close' in df.columns and not df['close'].isnull().all():
            # Calculate SMAs
            df.ta.sma(length=50, append=True) # SMA_50
            df.ta.sma(length=200, append=True) # SMA_200

            # Calculate EMA
            df.ta.ema(length=20, append=True) # EMA_20

            # Calculate RSI
            df.ta.rsi(length=14, append=True) # RSI_14

            # Calculate MACD
            df.ta.macd(fast=12, slow=26, signal=9, append=True) # MACD_12_26_9, MACDh_12_26_9, MACDs_12_26_9

            # Calculate Bollinger Bands
            df.ta.bbands(length=20, std=2, append=True) # BBL_20_2.0, BBM_20_2.0, BBU_20_2.0, BBB_20_2.0, BBP_20_2.0

            tech_data[symbol] = df
            print(f"Indicators calculated for {symbol}.")
            display(tech_data[symbol].dropna().tail()) # Show the last few rows with indicators (drop NaNs from initial calculation)
        else:
            print(f"'close' column missing or empty for {symbol}. Skipping indicator calculation.")
else:
    print("Stock data is empty. Skipping indicator calculation.")

### 2.2 Visualize Indicators

Plot the closing price along with the calculated indicators for each stock.

In [ ]:
if tech_data: # Check if tech_data was populated
    for symbol, df in tech_data.items():
        # Check if required columns exist before plotting
        required_cols = ['close', 'SMA_50', 'SMA_200', 'EMA_20', 'BBU_20_2.0', 'BBL_20_2.0', 'BBM_20_2.0',
                         'MACD_12_26_9', 'MACDs_12_26_9', 'MACDh_12_26_9', 'RSI_14']
        # Check if all required columns are present *and* have some non-NaN data
        if not all(col in df.columns and not df[col].isnull().all() for col in required_cols):
            print(f"Skipping visualization for {symbol} due to missing or all-NaN indicator columns.")
            continue
            
        print(f"Visualizing indicators for {symbol}...")

        # Create subplots: 3 rows for Price/MA/BBands, MACD, RSI
        fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                           vertical_spacing=0.05,
                           subplot_titles=(f'{symbol} Price, MAs, Bollinger Bands', 'MACD', 'RSI'),
                           row_heights=[0.6, 0.2, 0.2]) # Adjust row heights

        # --- Row 1: Price, MAs, Bollinger Bands ---
        # Price
        fig.add_trace(go.Scatter(x=df.index, y=df['close'], name='Close Price', line=dict(color='blue')), row=1, col=1)
        # MAs
        fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'], name='SMA 50', line=dict(color='orange', dash='dash')), row=1, col=1)
        fig.add_trace(go.Scatter(x=df.index, y=df['SMA_200'], name='SMA 200', line=dict(color='red', dash='dash')), row=1, col=1)
        fig.add_trace(go.Scatter(x=df.index, y=df['EMA_20'], name='EMA 20', line=dict(color='green', dash='dot')), row=1, col=1)
        # Bollinger Bands
        fig.add_trace(go.Scatter(x=df.index, y=df['BBU_20_2.0'], name='Upper Band', line=dict(color='gray', width=0.5)), row=1, col=1)
        fig.add_trace(go.Scatter(x=df.index, y=df['BBL_20_2.0'], name='Lower Band', line=dict(color='gray', width=0.5), fill='tonexty', fillcolor='rgba(128,128,128,0.1)'), row=1, col=1)
        fig.add_trace(go.Scatter(x=df.index, y=df['BBM_20_2.0'], name='Middle Band (SMA 20)', line=dict(color='gray', dash='dot', width=0.7)), row=1, col=1)


        # --- Row 2: MACD ---
        fig.add_trace(go.Scatter(x=df.index, y=df['MACD_12_26_9'], name='MACD Line', line=dict(color='purple')), row=2, col=1)
        fig.add_trace(go.Scatter(x=df.index, y=df['MACDs_12_26_9'], name='Signal Line', line=dict(color='magenta')), row=2, col=1)
        # Histogram (difference) - use Bar chart
        colors = ['green' if val >= 0 else 'red' for val in df['MACDh_12_26_9']]
        fig.add_trace(go.Bar(x=df.index, y=df['MACDh_12_26_9'], name='MACD Histogram', marker_color=colors), row=2, col=1)


        # --- Row 3: RSI ---
        fig.add_trace(go.Scatter(x=df.index, y=df['RSI_14'], name='RSI', line=dict(color='cyan')), row=3, col=1)
        # Add RSI overbought/oversold lines
        fig.add_hline(y=70, line_dash="dash", line_color="red", annotation_text="Overbought (70)", annotation_position="bottom right", row=3, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", annotation_text="Oversold (30)", annotation_position="bottom right", row=3, col=1)


        # --- Layout Updates ---
        fig.update_layout(
            title_text=f'{symbol} Technical Analysis',
            height=800, # Adjust height as needed
            legend_title_text='Indicators',
            xaxis_rangeslider_visible=False, # Hide range slider for cleaner look on subplots
            xaxis_showticklabels=True, xaxis2_showticklabels=True, xaxis3_showticklabels=True, # Ensure x-axis labels are shown if needed
            yaxis_title='Price', yaxis2_title='MACD', yaxis3_title='RSI'
        )
        # Ensure x-axis labels only show on the bottom plot
        fig.update_xaxes(showticklabels=False, row=1, col=1)
        fig.update_xaxes(showticklabels=False, row=2, col=1)
        fig.update_xaxes(showticklabels=True, row=3, col=1, title_text='Date')


        fig.show() # Display plot in Jupyter
else:
    print("Technical data dictionary is empty or symbols lack data. Skipping visualization.")

### 2.3 Technical Recommendation Logic

Define rules based on the indicators to generate Buy/Sell/Hold signals.
*   **RSI:** Buy < 30, Sell > 70
*   **MACD:** Buy on bullish crossover (MACD line crosses above signal line), Sell on bearish crossover.
*   **Moving Averages:** Buy when price > SMA50 and SMA50 > SMA200 (golden cross tendency), Sell when price < SMA50 and SMA50 < SMA200 (death cross tendency).
*   **Bollinger Bands:** Buy when price touches or breaks below lower band (potential reversal), Sell when price touches or breaks above upper band (potential pullback).

In [ ]:
# Dictionary to store recommendations
tech_recommendations = {}

if tech_data: # Check if tech_data was populated
    for symbol, df in tech_data.items():
        print(f"Generating technical recommendation for {symbol}...")
        
        # Drop rows with NaNs which usually occur at the start due to indicator lookback periods
        df_clean = df.dropna()
        
        if df_clean.empty:
            print(f"Not enough data for {symbol} after dropping NaNs. Skipping recommendation.")
            tech_recommendations[symbol] = "Not Enough Data"
            continue
            
        # Get the latest data point
        latest = df_clean.iloc[-1]
        signals = {'RSI': 0, 'MACD': 0, 'MA': 0, 'BB': 0} # 1 for Buy, -1 for Sell, 0 for Hold

        # --- Apply Rules ---
        # RSI
        if 'RSI_14' in latest and not pd.isna(latest['RSI_14']):
            if latest['RSI_14'] < 30:
                signals['RSI'] = 1 # Buy
            elif latest['RSI_14'] > 70:
                signals['RSI'] = -1 # Sell

        # MACD (Check crossover using the last two points of the cleaned data)
        if len(df_clean) > 1:
            prev = df_clean.iloc[-2]
            # Check if MACD and signal values are present and not NaN
            if all(k in latest and not pd.isna(latest[k]) for k in ['MACD_12_26_9', 'MACDs_12_26_9']) and \
               all(k in prev and not pd.isna(prev[k]) for k in ['MACD_12_26_9', 'MACDs_12_26_9']):
                # Bullish Crossover: MACD was below signal, now above
                if prev['MACD_12_26_9'] < prev['MACDs_12_26_9'] and latest['MACD_12_26_9'] > latest['MACDs_12_26_9']:
                    signals['MACD'] = 1 # Buy
                # Bearish Crossover: MACD was above signal, now below
                elif prev['MACD_12_26_9'] > prev['MACDs_12_26_9'] and latest['MACD_12_26_9'] < latest['MACDs_12_26_9']:
                    signals['MACD'] = -1 # Sell

        # Moving Averages (SMA 50 vs SMA 200 trend and Price vs SMA 50)
        if all(k in latest and not pd.isna(latest[k]) for k in ['SMA_50', 'SMA_200', 'close']):
            # Golden Cross tendency and price above SMA50
            if latest['SMA_50'] > latest['SMA_200'] and latest['close'] > latest['SMA_50']:
                 signals['MA'] = 1 # Buy
            # Death Cross tendency and price below SMA50
            elif latest['SMA_50'] < latest['SMA_200'] and latest['close'] < latest['SMA_50']:
                 signals['MA'] = -1 # Sell

        # Bollinger Bands
        if all(k in latest and not pd.isna(latest[k]) for k in ['BBL_20_2.0', 'BBU_20_2.0', 'close']):
            # Price touches or below lower band
            if latest['close'] <= latest['BBL_20_2.0']:
                signals['BB'] = 1 # Buy
            # Price touches or above upper band
            elif latest['close'] >= latest['BBU_20_2.0']:
                signals['BB'] = -1 # Sell

        # --- Combine Signals (Simple Average) ---
        valid_signals = [s for s in signals.values() if s is not None] # Assuming 0 is a valid signal (Hold)
        if len(valid_signals) > 0:
             final_signal_score = sum(valid_signals) / len(valid_signals)
        else:
             final_signal_score = 0 # Default to Hold if no signals

        if final_signal_score >= 0.5: # Threshold for Buy (Adjust as needed)
            recommendation = "Buy"
        elif final_signal_score <= -0.5: # Threshold for Sell (Adjust as needed)
            recommendation = "Sell"
        else:
            recommendation = "Hold"

        tech_recommendations[symbol] = recommendation
        print(f"{symbol} - Latest Clean Data Point:\n{latest}") # Print relevant latest data
        print(f"{symbol} - Individual Signals: {signals}")
        print(f"{symbol} - Signal Score: {final_signal_score:.2f}")
        print(f"{symbol} - Technical Recommendation: {recommendation}\n")

else:
    print("Technical data dictionary is empty. Skipping recommendation generation.")

## 3. Quantitative Analysis

Use statistical models to forecast price movements or classify future price direction.

### 3.1 Time Series Forecasting (ARIMA)

We'll use a simple ARIMA model to forecast the next day's closing price. Note: ARIMA requires careful parameter tuning (p, d, q) for optimal performance, which is beyond the scope of this basic example. We'll use placeholder parameters (e.g., p=5, d=1, q=0). Stationarity checks and parameter optimization (e.g., using ACF/PACF plots or auto_arima) are recommended for real-world applications.

In [ ]:
# Dictionary to store ARIMA forecasts
arima_forecasts = {}
arima_order = (5, 1, 0) # Example (p, d, q) order - NEEDS TUNING!

if not stock_data.empty:
    for symbol in symbols:
        print(f"Fitting ARIMA model for {symbol}...")
        # Select the 'close' price series for the symbol
        series = stock_data[symbol].dropna() # Use original data, ensure no NaNs
        
        if len(series) < (arima_order[0] + arima_order[2] + 10): # Check if enough data points
             print(f"Not enough data points for {symbol} to fit ARIMA({arima_order}). Skipping.")
             arima_forecasts[symbol] = None
             continue
             
        try:
            # Ensure the index has frequency information if possible (often daily)
            # If data is daily, uncommenting the next line might help
            # series = series.asfreq('B') # 'B' for business day frequency, adjust if needed
            # series = series.ffill() # Fill any gaps created by asfreq
            
            # Fit the ARIMA model
            # Use simple_differencing=False if d=0, True if d>0 (default is True)
            model = ARIMA(series, order=arima_order, enforce_stationarity=False, enforce_invertibility=False)
            model_fit = model.fit()
            
            # Forecast the next step
            forecast_result = model_fit.get_forecast(steps=1)
            forecast_value = forecast_result.predicted_mean.iloc[0]
            arima_forecasts[symbol] = forecast_value
            
            print(f"ARIMA forecast for {symbol} (next period): {forecast_value:.2f}")
            # Optional: Print model summary
            # print(model_fit.summary())
            
        except Exception as e:
            print(f"Error fitting ARIMA for {symbol}: {e}")
            arima_forecasts[symbol] = None # Indicate failure
            
else:
    print("Stock data is empty. Skipping ARIMA modeling.")

### 3.2 Quantitative Recommendation Logic

Generate recommendations based on the ARIMA forecast:
*   If forecast price > current price * 1.005 (e.g., 0.5% increase) -> Buy
*   If forecast price < current price * 0.995 (e.g., 0.5% decrease) -> Sell
*   Otherwise -> Hold

In [ ]:
# Dictionary to store quantitative recommendations
quant_recommendations = {}
forecast_change_threshold = 0.005 # 0.5% change threshold

if arima_forecasts and not stock_data.empty:
    for symbol, forecast in arima_forecasts.items():
        if forecast is None:
            print(f"No ARIMA forecast available for {symbol}. Skipping quantitative recommendation.")
            quant_recommendations[symbol] = "No Forecast"
            continue
            
        # Get the latest actual closing price
        latest_actual_price = stock_data[symbol].dropna().iloc[-1]
        
        print(f"Generating quantitative recommendation for {symbol}...")
        print(f"  Latest Actual Price: {latest_actual_price:.2f}")
        print(f"  ARIMA Forecast Price: {forecast:.2f}")
        
        recommendation = "Hold" # Default
        # Check for Buy signal
        if forecast > latest_actual_price * (1 + forecast_change_threshold):
            recommendation = "Buy"
        # Check for Sell signal
        elif forecast < latest_actual_price * (1 - forecast_change_threshold):
            recommendation = "Sell"
            
        quant_recommendations[symbol] = recommendation
        print(f"  Quantitative Recommendation: {recommendation}\n")
        
else:
    print("ARIMA forecasts or stock data are missing. Skipping quantitative recommendations.")

## 4. Combined Recommendation

Combine the signals from Technical and Quantitative analysis to provide a final recommendation. A simple approach is to average the signals (e.g., Buy=1, Hold=0, Sell=-1) or use a voting system.

In [ ]:
# Dictionary for final recommendations
final_recommendations = {}

# Define scoring for recommendations
score_map = {"Buy": 1, "Hold": 0, "Sell": -1, "Not Enough Data": 0, "No Forecast": 0}

print("--- Final Combined Recommendations ---")

for symbol in symbols:
    tech_rec = tech_recommendations.get(symbol, "Hold") # Default to Hold if missing
    quant_rec = quant_recommendations.get(symbol, "Hold") # Default to Hold if missing
    
    tech_score = score_map.get(tech_rec, 0)
    quant_score = score_map.get(quant_rec, 0)
    
    # Combine scores (simple average)
    # Handle cases where one analysis might be missing
    valid_scores = []
    if tech_rec not in ["Not Enough Data"]:
        valid_scores.append(tech_score)
    if quant_rec not in ["No Forecast"]:
        valid_scores.append(quant_score)
        
    if not valid_scores:
        combined_score = 0 # Default to Hold if both failed
    else:
        combined_score = sum(valid_scores) / len(valid_scores)
        
    # Determine final recommendation based on combined score
    final_rec = "Hold" # Default
    if combined_score >= 0.5: # Threshold for Buy
        final_rec = "Buy"
    elif combined_score <= -0.5: # Threshold for Sell
        final_rec = "Sell"
        
    final_recommendations[symbol] = final_rec
    
    print(f"Symbol: {symbol}")
    print(f"  Technical: {tech_rec} (Score: {tech_score})")
    print(f"  Quantitative: {quant_rec} (Score: {quant_score})")
    print(f"  Combined Score: {combined_score:.2f}")
    print(f"  FINAL RECOMMENDATION: {final_rec}\n")
    
# Display final recommendations in a table (optional)
final_df = pd.DataFrame.from_dict(final_recommendations, orient='index', columns=['Recommendation'])
display(final_df)